In [19]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.models import Sequential

In [3]:
from tensorflow.keras.layers import LSTM, Dense, Activation
# LSTM is the memory of our model

In [4]:
from tensorflow.keras.optimizers import RMSprop
# optimizer for compiling the model

In [5]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [6]:
# get the text of this file into our scripts, increase perfomance by only using lowercase leters
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

convert senntences to a numerial format, so that we can pass a numpy array into our network

we're going to find a way to convert a text on each character into a unique numerical representation and then this numerical representation back into the same character

We're going to train from part of the dataset to save time

In [7]:
text = text[300000:800000]

Create a character set that contains or the characters possible in the text

Set() filters out all the unique characters

In [8]:
characters = sorted(set(text))

create 2 dictionaries, so that we can convert these characters into numerical format and the numerical format back into the characters

eg{a:"1", f:"6"}  and {1:"a", 6:"f"}

c and i stand for character and index, enumerate basically assigns one number to each character

In [9]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

we're going to use 40 characters in order to predict the next character

seq length is the length of the sentence

step size is how many characters are we going to shift to the next sentence

In [10]:
SEQ_LENGTH = 40

STEP_SIZE = 3

create an empty list of sentences and an empty list of next characters

the next characters will be the target and the sentences will be the featues

so we load a sentence, and the result will be the following character

In [11]:
sentences = []
next_characters = []

fill up that list based on the sequence length and the step size

this loop runs from the beginning of the text until the last part of the text where we can read a whole sequence length and always use a step size of 3 in between

sentences append the part of the text that reaches from i up until i+SEQ_LENGTH

if the seq length is 5 we're getting the character zero up until 4 and then the character with the index 5 is the next character which is what we wanna have

In [12]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i+SEQ_LENGTH])                  # i is the starting position that we're at now and the seq length is basically the length of the sentence
                                                             # i+SEQ_LENGTH is the sequence that we wanna add to the sentences
    next_characters.append(text[i+SEQ_LENGTH])               # if the seq .....^

    so now we have the feature data and the target data but it is stil in string format, we need a numerical format

    convert the training data that we just created into a numpy array

create an numpy array full zeros, and the shape shall be the length of the sentences, so basically how many sentences there are, times the seq length (length of the sentences) times the 

amount of possible characters. and the data type will be boolean

    whenever in a specific sentence, at a specific position, a certain character occurs, we're going to set that to true or 1. And all the other values will remain 0
    
    eg is we have sentence number 5 and at a position number 7, we have the character with the enumeration 8, we're going to say 5 7 8 = 1 because their character occurs
    
    so in this format we're going to pass the training data to a neural network
    
    at sencence number 5 , the next character is the character with the enumeration 8

In [13]:
# input
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)                # len of sentences  = the amount of sentences we have
# target
y = np.zeros((len(sentences), len(characters)), dtype=bool)           

now we just need to fill up the arrays

we're going to fill them with two for loops

we're running one for loop over all the sentences, we take all the sentences and assign an index to them

for each of those sentences, we then again enumerate every single character in the sentence

so we say t, which is the character of the sentence, or the position of the sentence character, we say basically index that as well

and then we say, the position in the x, sentence number i, at position number t, and character number whatever (so we take the charcter at this position), convet it into a numerical format. This position is set 1

because that character occurs at that position in that sentence, therefore it is true or 1

In [14]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1   #training data
    y[i, char_to_index[next_characters[i]]] = 1 #output

# Build the neural net 

### Train the model

In [15]:
model = Sequential()
model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))          #LSTM = short term memory of our neural net
model.add(Dense(len(characters)))                                      #Dense Layer, has as many neurons as we have possible characters
model.add(Activation('softmax'))                                       #Activation Layers, softmax scales the value so they add up to 1

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 [==============================] - 86s 129ms/step - loss: 2.3955
Epoch 2/4
651/651 [==============================] - 84s 129ms/step - loss: 1.9239
Epoch 3/4
651/651 [==============================] - 82s 127ms/step - loss: 1.7425
Epoch 4/4
651/651 [==============================] - 84s 129ms/step - loss: 1.6479


### save the model so we don't have to train it again

load it whenever needed


In [17]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


We're going to load it like this

model = tf.keras.models.load_model('textgenerator.model')

or

from tensorflow import keras
model = keras.models.load_model('path/to/location')

### Load the model

load it, you can delete the train the model code, but we'll keep it here

In [21]:
model = tf.keras.models.load_model('textgenerator.model')

### Get predictions of the model and convert these predictions into text generations

what our model does is take a sequence and predic the next character, we need to make it generate text

for this we need a helper function copied from the official keras tutorial

### Helper function

This helper function called sample is copied from the official Keras [tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/).

It basically just picks one of the characters from the output. As parameters it takes the result of the prediction and a temperature. This temperature indicates how risky the pick shall be. If we have a high temperature, we will pick one of the less likely characters. A low temperature will cause a conservative choice.

In [22]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Text generation

In [26]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [28]:
print('-------0.2-----------')
print(generate_text(300, 0.2))
print('-------0.4-----------')
print(generate_text(300, 0.4))
print('-------0.6-----------')
print(generate_text(300, 0.6))
print('-------0.8-----------')
print(generate_text(300, 0.8))
print('--------1.0-----------')
print(generate_text(300, 1.0))

-------0.2-----------
here;
then with directions to repair to the streak stand.

duke of york:
why shall the streak to the sen to me the sen
and to the strake thy dead to the words,
and streak streak streak stand to the son the traither;
and the reaster the server to my lord.

gloucester:
what i have shall stay the shall stand in his son the seess heart,
and m
-------0.4-----------
betray the best!
turn then my freshest reall him his the earth,
and we stand the simpers to the mook,
and seed for that not the stret wilt the man stort,
that the reasth i will the arm so the were me
the searth my love and the great to to me.

clifford:
i may to the sees of condeds the earth,
and in the sen of his sends and life.

romeo:

-------0.6-----------
ld coat,
razed out my imprese, leaving may and fries.

king richard ii:
no for have see stand any in right him have be of true on edward deed.

camillo:
what had i his oress try grief or to my kinds,
i am is you weep her spares to thou stee have men,
s